### 2.1.2019
### Funkce na stahování akcií
### Využití jednak pro Revolut, ale hlavně pro diplomku - stahování dat

#### 5.1. už mám returns, plot řady i returns, load faktorů z excelu EU/US
#### 15.1. dlouho nic moc, dnes jen opravena data faktorů v excelu
#### 19.1. Dnes udělána plošná regrese, vtupy jsou proměnná, počet faktorů, split.
#### 21.-22.1. Loading CSV - header, formát data, loading podle inputu
* Řeším lepší gruppen plot
* kontrola merge, že mi dobře doléhá datum stocku a faktorů
* kontrola dropna() že to funguje dobře a že nemám NANs uvnitř
* Teď pracuju s Daily, možná budou lepší Monthly !! nebude jich 7.000 ale jen 250

Je pravda, že jde o to, zjistit jestli cena akcie rost kvůli rostoucí hodnotě podkladu (kvalita firmy), nebo roste cena jen kvůli zvýšenímu zájmu, což může být samovyživovací bublina?


# Obsah

1. Import Packages
2. Scrapping Stocks + Visualize
3. ARIMA
4. Factors
5. Regression
6. LSTM - simple time series
7. LSTM - time series + factors

# 1. Import Packages

In [1]:

# 1.) Obecné
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure  
import numpy as np
import math
from pandas import DataFrame
from math import sqrt
from datetime import datetime
import csv

# 2.) Scrapping
import requests # for robots check
import lxml.html as lh
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains      # ?????
from time import sleep

# 3.) REGRESE
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# 4.) LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


C:\Users\jiri.zelenka\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# print(requests.get("https://finance.yahoo.com/robots.txt").text)

# 2. Scrapping Stocks + Visualize

### Data Download

In [ ]:


def data_dwn(stocks):

    browser = webdriver.Chrome()

    for i in stocks:

        url = "https://finance.yahoo.com/quote/" + str(i) + "/history?period1=10104800&period2=1577919600&interval=1d&filter=history&frequency=1d"
        browser.get(url)

        # Tohle mi odklikne OK warning a pustí mě to na stránku

        try:
            play = browser.find_elements_by_xpath('/html/body/div/div/div/form/div/button[2]')[0]
            hover = ActionChains(browser).move_to_element(play)
            hover.perform()
            play.click()

        except:
            pass
        
        sleep(5)    # 3 sec bylo málo .. 5sec mi zvládlo už 8/9 místo 5/9 bez chyby
        play = browser.find_elements_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[2]/span[2]')[0]
        hover = ActionChains(browser).move_to_element(play)
        hover.perform()
        play.click()
        
#############################################################################

akcie = ["AAPL", "AMZN", "FB", "GOOG","NFLX", "NKE", "SBUX", "TSLA", "XIACF"]

#akcie = ["ATVI", "BABA", "AAL", "T","BIDU", "BB", "BKNG", "CAJ", "CSCO"]

#akcie = ["CLDR", "KO", "CROX", "DELL", "DAL", "DBX", "EBAY", "EA", "XOM"]


#data_dwn(akcie)

### Data import + merge

In [ ]:


def load_all_csv(path, price, stocks):
    
    data = pd.DataFrame()
    
    for i in stocks:
    
        df = pd.read_csv(path + i + '.csv')
        df = df[["Date", price]]
        df = pd.DataFrame(df, columns= ['Date', price])
        df.columns = ["Date " + i , i]
        df = df.iloc[::-1]
# Otočení pořadí, aby se matchovaly nejnovější hodnoty
        df.reset_index(drop = True,inplace=True)
        
        data = pd.concat([data, df], axis=1)
        df = data
    
    return df

#############################################################################

cesta = "C:/Users/jiri.zelenka/Downloads/"
cena = "Close"

#data = load_all_csv(cesta, cena, akcie)

### Date + otočení zpět

In [ ]:


def date_all(df):

    df["Date"] = np.nan
    df["Date2"] = np.nan

    for i in akcie:
        if df["Date " + i].isnull().iloc[len(df)-1] + 1 == 1:

            df["Date"] = df["Date " + i]

    # Zajímavý poznatek:

    # pokud testuju value, tak 0= false, 1 = true
    # ale pokud už k výsledku něco přičtu a mám třeba 1 a 2, tak to 1 už ale není TRUE !!!

    df["Date2"] = df["Date"]

    df = df.iloc[:, 1:2+len(akcie)*2:2]
    
    df = df.iloc[::-1]
# Otočení zpátky
    df.reset_index(drop = True,inplace=True)

    return df

#############################################################################

#data = date_all(data)

### Difference = Return

In [ ]:


def diff_returns(df):
    for i in akcie:
        df["Returns " + i] = df[i].shift(+1)
        df["Returns " + i] = (df[i]/df["Returns " + i]) -1
        
        # Elegantní, rychlé, bez řešení řádků - Shift a pak celý loupec dělený celým sloupcem
        
    return df

#############################################################################

#data = diff_returns(data)

### Basic Plot

In [ ]:

####!!!!
# Předělat If, else - If else Pass na elif
#### !!!

def basic_plots(df,cols):

    fig, axs = plt.subplots(3, 3)

    ## Nedaří se mi zautmoatizovat, ani tvar ani plotování
    
    if cols == "akcie":
    
        axs[0, 0].plot(df[akcie[0]], 'tab:blue')
        axs[0, 0].set_title(akcie[0])
        axs[0, 1].plot(df[akcie[1]], 'tab:orange')
        axs[0, 1].set_title(akcie[1])
        axs[0, 2].plot(df[akcie[2]], 'tab:green')
        axs[0, 2].set_title(akcie[2])
        axs[1, 0].plot(df[akcie[3]], 'tab:red')
        axs[1, 0].set_title(akcie[3])
        axs[1, 1].plot(df[akcie[4]], 'tab:blue')
        axs[1, 1].set_title(akcie[4])
        axs[1, 2].plot(df[akcie[5]], 'tab:orange')
        axs[1, 2].set_title(akcie[5])
        axs[2, 0].plot(df[akcie[6]], 'tab:green')
        axs[2, 0].set_title(akcie[6])
        axs[2, 1].plot(df[akcie[7]], 'tab:red')
        axs[2, 1].set_title(akcie[7])
        axs[2, 2].plot(df[akcie[8]], 'tab:blue')
        axs[2, 2].set_title(akcie[8])


        fig = plt.gcf()
        fig.set_size_inches(18.5, 10.5)
        
    else:
            
        if cols == "returns":

            axs[0, 0].plot(df["Returns " + akcie[0]], 'tab:blue')
            axs[0, 0].set_title("Returns " + akcie[0])
            axs[0, 1].plot(df["Returns " + akcie[1]], 'tab:orange')
            axs[0, 1].set_title("Returns " + akcie[1])
            axs[0, 2].plot(df["Returns " + akcie[2]], 'tab:green')
            axs[0, 2].set_title("Returns " + akcie[2])
            axs[1, 0].plot(df["Returns " + akcie[3]], 'tab:red')
            axs[1, 0].set_title("Returns " + akcie[3])
            axs[1, 1].plot(df["Returns " + akcie[4]], 'tab:blue')
            axs[1, 1].set_title("Returns " + akcie[4])
            axs[1, 2].plot(df["Returns " + akcie[5]], 'tab:orange')
            axs[1, 2].set_title("Returns " + akcie[5])
            axs[2, 0].plot(df["Returns " + akcie[6]], 'tab:green')
            axs[2, 0].set_title("Returns " + akcie[6])
            axs[2, 1].plot(df["Returns " + akcie[7]], 'tab:red')
            axs[2, 1].set_title("Returns " + akcie[7])
            axs[2, 2].plot(df["Returns " + akcie[8]], 'tab:blue')
            axs[2, 2].set_title("Returns " + akcie[8])
            
            fig = plt.gcf()
            fig.set_size_inches(18.5, 10.5)
            
        else:
            pass
        
#############################################################################

#basic_plots(data, "akcie")
#basic_plots(data, "returns")

In [ ]:
# Mixed plot

#for i in akcie:

#    display(plt.plot(data[i].dropna()))


### Statistiky řad

In [ ]:
# data.describe()

## Load + Date + Plot + Stat + Difference + Plot

In [168]:

def load_date_plot_stat(path, price, stocks):
    
        df = diff_returns(date_all(load_all_csv(cesta, cena, akcie)))
            
        display(df.describe())
        basic_plots(df, "akcie")
        basic_plots(df, "returns")
        return df

    # Zajímavé: {display, plot, return} - pokud spustím fci přímo, zobrazí se všechny tři věci
    # Pokud spustím fuknkci jako var = fce(), tak return se nezobrazí, ale právě uloží do proměnné -> viz níže
    
#############################################################################   

cesta = "C:/Users/jiri.zelenka/Desktop/Python/Akcie_diplomka/stocks/"
cena = "Close"

#akcie = ["AAPL", "AMZN", "FB", "GOOG","NFLX", "NKE", "SBUX", "TSLA", "XIACF"]
akcie = ["ATVI", "BABA", "AAL", "T","BIDU", "BB", "BKNG", "CAJ", "CSCO"]
#akcie = ["CLDR", "KO", "CROX", "DELL", "DAL", "DBX", "EBAY", "EA", "XOM"]

#data = load_date_plot_stat(cesta, cena, akcie)
display(data.head())
display(data.tail())

NameError: name 'diff_returns' is not defined

### Save the data

In [ ]:
# data.to_excel(r"data "+ akcie[0]+ " - " + akcie[len(akcie)-1] + " total_" + str(len(akcie)) + ".xlsx")

# 3. ARIMA

In [ ]:
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_pacf

from statsmodels.tsa.arima_model import ARIMA

In [ ]:
def autocorr_plots(df):

    autocorrelation_plot(data["XIACF"].dropna())
    autocorrelation_plot(data["NKE"].dropna())


    fig = plt.gcf()
    fig.set_size_inches(10.5, 6.5)

    
#autocorr_plots(data)


In [ ]:

# Datacamp:

data["Returns CAJ"].dropna().autocorr()


In [ ]:
#autocorrelation_plot(data["AMZN"].dropna())

In [ ]:
#autocorrelation_plot(data["Returns AMZN"].dropna())

In [ ]:
#autocorrelation_plot(data["AAPL"].dropna())

In [ ]:
#autocorrelation_plot(data["Returns AAPL"].dropna())

In [ ]:
#autocorrelation_plot(data["FB"].dropna())

In [ ]:
#autocorrelation_plot(data["Returns FB"].dropna())

In [ ]:
#autocorrelation_plot(data["GOOG"].dropna())

In [ ]:
#autocorrelation_plot(data["Returns GOOG"].dropna())

In [ ]:
#autocorrelation_plot(data["NFLX"].dropna())

In [ ]:
#autocorrelation_plot(data["Returns NFLX"].dropna())

In [ ]:
#autocorrelation_plot(data["NKE"].dropna())

In [ ]:
#autocorrelation_plot(data["Returns NKE"].dropna())

In [ ]:
#autocorrelation_plot(data["SBUX"].dropna())

In [ ]:
#autocorrelation_plot(data["Returns SBUX"].dropna())

In [ ]:
#autocorrelation_plot(data["TSLA"].dropna())

In [ ]:
#autocorrelation_plot(data["Returns TSLA"].dropna())

In [ ]:
#autocorrelation_plot(data["XIACF"].dropna())

In [ ]:
#autocorrelation_plot(data["Returns XIACF"].dropna())

In [ ]:
model = ARIMA(data["CAJ"].dropna(), order=(1,1,1))    #(1,1,1) i (2,1,2) dává dost signifikantní výsledky, vyšší čísla už sekají výpočty
model_fit = model.fit(disp=0)
print(model_fit.summary())

In [ ]:
residuals = DataFrame(model_fit.resid)
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

# 4. Factors

### Scrapping factors

### Loading CSV with correct header

#### Not working for me:

In [129]:
################################################################################################################################################x
import csv
reader = csv.DictReader(open("C:/Users/jiri.zelenka/Desktop/Python/Akcie_diplomka/factors/Five_factors_EU_daily.csv"))
"""if reader.fieldnames[0][0] == "T":
    df7 = pd.read_csv("C:/Users/jiri.zelenka/Desktop/Python/Akcie_diplomka/factors/Five_factors_EU_daily.csv", skiprows = 1)
else:
    pass
df7
"""

print(reader.fieldnames[0])


"""
i=1

while reader.fieldnames[0][0] != ",":
    i = i+1
    print(i)"""

## kjk
################################################################################################################################################x
with open("C:/Users/jiri.zelenka/Desktop/Python/Akcie_diplomka/factors/Five_factors_EU_daily.csv", 'r') as f:
    d_reader = csv.DictReader(f)

    #get fieldnames from DictReader object and store in list
    headers = d_reader.fieldnames

    for line in d_reader:
        #print value in MyCol1 for each row
        print(line)
################################################################################################################################################x
with open("C:/Users/jiri.zelenka/Desktop/Python/Akcie_diplomka/factors/Five_factors_EU_daily.csv", 'r') as f:
    #you can eat the first line before creating DictReader.
    #if no "fieldnames" param is passed into
    #DictReader object upon creation, DictReader
    #will read the upper-most line as the headers
    f.readline()

    d_reader = csv.DictReader(f)
    for row in d_reader:
        print(row)
################################################################################################################################################x
df7 = pd.read_csv("C:/Users/jiri.zelenka/Desktop/Python/Akcie_diplomka/factors/Five_factors_US_daily.csv", skiprows = 2, header = None)
display(type(df7.iloc[0].iloc[0])==str)
display(df7[0][0])
display(df7.iloc[0].iloc[0])
df7.head()

# Kombinace skip = 12, header = None mi nefunguje na Momentum_US_daily, protože tam pak [0][0] nemám žádný string, jen na [1][0]
# jinak skiprows + header = None fungovala snad na všechno.
# Pro změnu kombinace úplně bez headeru mi zase nefunguje na monthly data ,které jsou celé string.

################################################################################################################################################
i = 0
path = "C:/Users/jiri.zelenka/Desktop/Python/Akcie_diplomka/factors/Momentum_US_daily.csv"

while True :
    try:
        df = pd.read_csv(path, skiprows = i, header = None)
        
        if df.iloc[0].iloc[0] in ["RF", "CMA"]:          # Jeden z problémů: Nejde použít STRING, protože v EU_monthly jsou i čísla jako stringy...// v Momentum taky.. ?
            i = i+1                                  # Takže budu muset zůstat u detekce RF a CMA .... smutné ale je to tak
        else:
            data = pd.read_csv(path , skiprows = i, header = None)
            break
    except:
        i = i+1
        
data.head(15)

False

nan

nan

,0,1,2,3,4,5,6
0,NaN,Mkt-RF,SMB,HML,RMW,CMA,RF
1,19630701.0,-0.67,0.00,-0.32,-0.01,0.15,0.012
2,19630702.0,0.79,-0.27,0.27,-0.07,-0.19,0.012
3,19630703.0,0.63,-0.17,-0.09,0.17,-0.33,0.012
4,19630705.0,0.40,0.08,-0.28,0.08,-0.33,0.012


#### Finally working for me:

In [230]:

def find_header_and_load(full_path):

    pos= 0
    oldpos = None

    with open(full_path, 'r') as fp:

        while pos != oldpos:  # make sure we stop reading, in case we reach EOF
            line= fp.readline()
            if line.startswith(","):
                fp.seek(pos)                  
                break
            oldpos = pos 
            pos = fp.tell()    # renenber this position as sthe start of the next line ############x ?? Je to podobnéjako +1 ??

        df = pd.read_csv(fp)
        
    return df

# Seek and Tell:
# https://www.youtube.com/watch?v=SEURXzTgvKI

# Odkaz od Davida:
# https://stackoverflow.com/questions/14881885/python-pandas-read-csv-file-with-variable-preamble-length?fbclid=IwAR166rOeOxm16qJ7NWoB_tHz1M9JgfCIlORcsY3JaT0NNc5HYMObjRTzhpc

#############################################################################

fullpath = "C:/Users/jiri.zelenka/Desktop/Python/Akcie_diplomka/factors/Momentum_US_daily.csv"

data = find_header_and_load(fullpath)
data.head()

,Unnamed: 0,Mom
0,19261103,0.57
1,19261104,-0.52
2,19261105,1.15
3,19261106,-0.04
4,19261108,-0.01


In [229]:

def format_date_daily(df):

    df = df.rename(columns={'Unnamed: 0': 'Date'})
    df['Date'] = df['Date'].apply(str)
    
    try:                                                                 

        df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x,'%Y%m%d'))      
   
    except:
        df = df[0:len(df)-1]                                                     #### Kvůli tomu, že některé soubory měly na konci řádek s copyrightem tkerý je třeba umazat
        df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x,'%Y%m%d'))  
        
    return df

        # Byl to boj, různé verze strtime strftime, dále taky problem zase s konverzí stringu, snaha zbavit se konce toho /no atd., nakonec po apply to vyšlo krásně, a samy naskákaly pomlčky do datumů. :)
#############################################################################

data = data
data = format_date_daily(data)
data.tail()

,Date,Mom
24516,2019-11-22,-1.02
24517,2019-11-25,-0.36
24518,2019-11-26,0.93
24519,2019-11-27,-0.24
24520,2019-11-29,-0.01


In [231]:
def format_date_monthly(df):

    df = df.rename(columns={'Unnamed: 0': 'Date'})
    df['Date'] = df['Date'].apply(str)
    
    try:                                                                 

        df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x,'%Y%m'))      ###### Buď je blbě tenhle formát, nebo to blbne kvůli těm dalším posunutý mřádkům ... 
   
    except:
        df = df[0:len(df)-1]                                                     #### Kvůli tomu, že některé soubory měly na konci řádek s copyrightem tkerý je třeba umazat
        df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x,'%Y%m'))  
        
    return df

#############################################################################
data = data
data = format_date_monthly(data)
data.tail()

ValueError: unconverted data remains: 03

### Loading specific EXCEL XXX CSV and preparing

In [224]:

def factors_load(path, country, freq):
    
    if country == "EU":
        i = "EU"
    elif country == "US":
        i = "US"
    else:
        pass

    if freq == "daily":
        j = "daily"
    elif freq == "monthly":
        j = "monthly"
    else:
        pass


    fullpath1 = path + "Five_factors_" + i + "_" + j +".csv"
    fullpath2 = path +  "Momentum_" + i + "_" + j +".csv"
    fullpath3 = path + "Three_factors_" + i + "_" + j +".csv"
    
    if freq == "daily":
        df1 = format_date_daily(find_header_and_load(fullpath1))
        df2 = format_date_daily(find_header_and_load(fullpath2))
        df3 = format_date_daily(find_header_and_load(fullpath3))

        
    if freq == "monthly":
        df1 = format_date_monthly(find_header_and_load(fullpath1))
        df2 = format_date_monthly(find_header_and_load(fullpath2))
        df3 = format_date_monthly(find_header_and_load(fullpath3)) 
    
    return df1, df2, df3

#############################################################################

path = "C:/Users/jiri.zelenka/Desktop/Python/Akcie_diplomka/factors/"
factors = factors_load(path, "EU", "daily")

three = factors[2]
four = factors[1]
five = factors[0]


################
# !!!!!!!!!!!!!!!!! Porblém je u UUS i EU monthly - tam to chápu, nedořešil jsem formáty pomlček

#

In [225]:
display(three.head())
display(four.head())
display(five.head())

display(three.tail())
display(four.tail())
display(five.tail())


,Date,Mkt-RF,SMB,HML,RF
0,1990-07-02,0.99,0.05,-0.53,0.03
1,1990-07-03,0.33,-0.12,-0.03,0.03
2,1990-07-04,0.24,-0.01,-0.14,0.03
3,1990-07-05,-0.64,0.19,0.06,0.03
4,1990-07-06,0.08,-0.19,0.14,0.03


,Date,WML
0,1990-11-01,0.24
1,1990-11-02,0.32
2,1990-11-05,0.47
3,1990-11-06,0.56
4,1990-11-07,0.34


,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,1990-07-02,0.99,0.01,-0.53,0.41,-0.25,0.03
1,1990-07-03,0.33,-0.09,-0.03,0.05,0.27,0.03
2,1990-07-04,0.24,0.01,-0.14,-0.10,0.23,0.03
3,1990-07-05,-0.64,0.20,0.06,-0.32,0.08,0.03
4,1990-07-06,0.08,-0.24,0.14,0.02,0.13,0.03


,Date,Mkt-RF,SMB,HML,RF
7626,2019-09-24,0.12,0.09,-0.63,0.01
7627,2019-09-25,-1.27,-0.33,0.51,0.01
7628,2019-09-26,0.34,-0.48,-0.39,0.01
7629,2019-09-27,0.54,-0.19,0.28,0.01
7630,2019-09-30,-0.02,-0.01,-0.03,0.01


,Date,WML
7538,2019-09-24,1.11
7539,2019-09-25,-0.17
7540,2019-09-26,0.89
7541,2019-09-27,-0.70
7542,2019-09-30,-0.03


,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
7626,2019-09-24,0.12,0.06,-0.63,0.13,-0.23,0.01
7627,2019-09-25,-1.27,-0.32,0.51,-0.32,0.21,0.01
7628,2019-09-26,0.34,-0.52,-0.39,0.08,-0.27,0.01
7629,2019-09-27,0.54,-0.17,0.28,0.00,0.15,0.01
7630,2019-09-30,-0.02,-0.01,-0.03,-0.05,-0.19,0.01


In [ ]:
length = min(len(three), len(four), len(five))


"""for i in (three, four, five):
    
    i = i.iloc[::-1]
    i = i[0:length]
    i = i.iloc[::-1]
    
   """

three = three.iloc[::-1]
three = three[0:length]
three = three.iloc[::-1]

four = four.iloc[::-1]
four = four[0:length]
four = four.iloc[::-1]

five = five.iloc[::-1]
five = five[0:length]
five = five.iloc[::-1]

for i in (three, four, five):
    i = i[0:length]
    print(len(i))
    
 ### Seříznutí na stejnou délku pro všechny tři verze faktorů !!! 


    
four = pd.concat([three, four["WML"]], axis=1)
four = four[["Mkt-RF", "SMB", "HML","WML", "RF"]]
#four´factor je  prozatím založeno na 3 faktoru



## Save the data

In [ ]:
#three.to_excel(r"three.xlsx")
#four.to_excel(r"four.xlsx")
#five.to_excel(r"five.xlsx")

# 5. Regression

In [ ]:

def prepare_OLS(stocks, data, num_factors, split):
    
    if num_factors == "three":
        factors = [ "Mkt-RF", "SMB", "HML"]
        num_factors = three
    else:
        if num_factors == "four":
            factors = [ "Mkt-RF", "SMB", "HML", "WML"]
            num_factors = four
        else:
            if num_factors == "five":
                factors = [ "Mkt-RF", "SMB", "HML", "RMW", "CMA"]
                num_factors = five
            else:
                pass             ###### zE ZAHÁDNÝHO DŮVODU MI to nešlo přes porovnání Parametru s dataframeme (three, four, five..), musel jsem teda přes string a návrat zpět
    
    
    
    for i in stocks:
        returns =  data[["Date2","Returns " + i ]]
        returns = returns.set_index('Date2')

        merged = returns.merge(num_factors, left_index=True, right_index=True, how='inner')
        data2 = merged.dropna()                    ### Tady byl jeden problém, proč mi to házelo hodně nul v R squared
 #       data2 = data2[len(data2)-100:len(data2)]    ############ Pokus na konci dat, abych viděl regresi na závěru - získal jsem i R2 20% konečně !!
            
        data2["EXCESS"] = data2["Returns " + i] - data2["RF"]
        
        X = data2[factors]      ################ Tady to musím poladit asi IFem ,,,
        y = data2[["EXCESS"]]
        
        #plt.plot(y[len(y)-10:len(y)])            ###########x Tady pořád mi jde vykreslovat jen pár hodnot
        #plt.show()                             ############# Subploty ještě pořád nevím jak dodělám
        

        split = split

        train_X = X[0:round(split*len(X))]
        train_y = y[0:round(split*len(y))]

        test_X = X[round(split*len(X)):]
        test_y = y[round(split*len(y)):]
        
        model = sm.OLS(train_y, train_X).fit()
        predictions = model.predict(test_X) 
        #display(model.summary())                  ###### Celé summary asi nepotřebuju
        display("R2: ", model.rsquared)
        rms = sqrt(mean_squared_error(test_y, predictions))
        display("RMSE: ", rms)
        display("Length: ", len(data2))                                                 
            
        plt.plot(y[len(y)-100:len(y)])
        plt.plot(predictions[len(predictions)-100:len(predictions)], color='red')

        #fig = plt.gcf()
        #fig.set_size_inches(10, 8)

        plt.show()

In [ ]:
akcie = ["AAPL", "AMZN", "FB", "GOOG","NFLX", "NKE", "SBUX", "TSLA", "XIACF"]
#akcie = ["ATVI", "BABA", "AAL", "T","BIDU", "BB", "BKNG", "CAJ", "CSCO"]
#akcie = ["CLDR", "KO", "CROX", "DELL", "DAL", "DBX", "EBAY", "EA", "XOM"]


In [ ]:
prepare_OLS(stocks = akcie, data = data, num_factors = "five", split = 0.8)

# Pozn: R2 vychází dobře jen když řeším třeba posledních 100 záznamů,  (38 %)
# Když jedu na celý dataset, většinou je R2 kolem 7 % ...


# 6. LSTM - simple time series

In [ ]:
np.random.seed(7)

y = y.values
y = y.astype('float32')

y = y.reshape(-1, 1)   

# normalize the dataset
scaler = MinMaxScaler(feature_range=(-1, 1))
y = scaler.fit_transform(y)

# split into train and test sets

splitty = 0.8

train_size = int(len(y) * splitty) 
test_size = len(y) - train_size    
train = y[0:train_size, :]
test = y[train_size:len(y), :] 
display(len(train), len(test))

def create_dataset(dataset, look_back=10):        
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):   
        a = dataset[i:(i+look_back), 0]         
        dataX.append(a)                         
        dataY.append(dataset[i + look_back, 0]) 
    return np.array(dataX), np.array(dataY)
                                             

# reshape into X=t and Y=t+1
look_back = 5
train_X, train_y = create_dataset(train, look_back)
test_X, test_y = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))


# create and fit the LSTM network
model = Sequential()
model.add(LSTM(8, input_shape=(1, look_back)))
model.add(Dense(4))    # Přidána nová vrstva...
model.add(Dense(1))    
model.compile(loss='mean_squared_error', optimizer='adam')

# Activation = RELU?
#8 = ??
# v druhém Lstm nemám druhou vrstvu 
# taky tam mám batch 72 místo 1 ...



model.fit(train_X, train_y, epochs=100, batch_size=1, verbose=2)

### # make predictions
trainPredict = model.predict(train_X)
testPredict = model.predict(test_X)



# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
train_y = scaler.inverse_transform([train_y])
testPredict = scaler.inverse_transform(testPredict)
test_y = scaler.inverse_transform([test_y])

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(train_y[0], trainPredict[:,0]))
print('Train Score: %.6f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(test_y[0], testPredict[:,0]))
print('Test Score: %.6f RMSE' % (testScore))


                                                                   
# shift train predictions for plotting
"""trainPredictPlot = np.empty_like(y)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict     # úsek [5:37] naplň 32 hodnotami predikce
"""
# shift test predictions for plotting
testPredictPlot = np.empty_like(y)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(y)-1, :] = testPredict 

# plot baseline and predictions
plt.plot(scaler.inverse_transform(y))
"""plt.plot(trainPredictPlot) """

plt.plot(testPredictPlot, color = "green")    #LSTM - začíná o pět tahů později kvůli "lagu"
plt.plot(predictions, color='red')            #Regrese


plt.show()



# Detail na konec setu

plt.plot(scaler.inverse_transform(y))
plt.plot(testPredictPlot, color = "green")    #LSTM - začíná o pět tahů později kvůli "lagu"
plt.plot(predictions, color='red')            #Regrese

plt.xlim(len(X)*splitty-5,len(X))   
plt.ylim(-0.12, 0.11)

fig = plt.gcf()
fig.set_size_inches(10, 8)
# fig.savefig('test2png.png', dpi=100)

plt.show()


# 7. LSTM - time series + factors